In [ ]:
from polus.pipelines.build import build_workflow, generate_compute_workflow
from pathlib import Path
import logging
import sys


logging.basicConfig(stream=sys.stdout, level=logging.DEBUG, format="%(asctime)s %(message)s")
logging.getLogger("polus.pipelines")



#workflow = build_workflow(Path("../config/download/BBBC/BBBC001_download.yaml"))
workflow = build_workflow(Path("../config/process/BBBC/BBBC001_process.yaml"))

shapes = solara.reactive(workflow)

def updateStep(value):
    text.set(f"Step updated! {value}")

def do_generate_compute_workflow():

    text.set("run workflow")
    compute_workflow = generate_compute_workflow(workflow)

def create_ui_element(input):
    title = input.name
    input_type = input.inp_type.__name__
    if  input_type == "str":
        solara.InputText(title, value=input.value, continuous_update=False, on_value=updateStep)
    elif input_type == "Path":
        if not input.linked:
            solara.InputText(title, value=input.value.as_posix(), continuous_update=False, on_value=updateStep)
    elif input_type == "bool":
        solara.Checkbox(label=title, value=input.value)

for step in workflow.steps:
    print(step.cwl_name)
    for input in step.inputs:
        print(input)
        create_ui_element(input)

import solara

text = solara.reactive("")
continuous_update = solara.reactive(True)

@solara.component
def Page():
    for step in workflow.steps:
        plugin_name = step.cwl_name
        solara.Markdown(f"##{plugin_name}")
        for input in step.inputs:
            create_ui_element(input)
    with solara.Row():
        solara.Button("Run Workflow", on_click=do_generate_compute_workflow)
        solara.Button("Reset", on_click=lambda: text.set("reset workflow config"))
    solara.Markdown(f"**Status**: {text.value}")

display(Page())
